In [1]:
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import talib
import data
import feature_engineering

warnings.simplefilter(action='ignore', category=FutureWarning)

# Read Prices

In [2]:
df_equity = data.read_excel_sheets()
df_equity['OpenTime'] = pd.to_datetime(df_equity['OpenTime'])
df_equity = df_equity.set_index('OpenTime') 

list_equity = df_equity.Symbol.unique().tolist()
print(f"\nEquity:\n{list_equity}")

crypto = 'data_crypto.xlsx'
df_crypto = (
    pd.read_excel(crypto)
    .drop(columns=['Unnamed: 0'])
)

df_crypto['OpenTime'] = pd.to_datetime(df_crypto['OpenTime'])
df_crypto = df_crypto.set_index('OpenTime') 

list_crypto = df_crypto.Symbol.unique().tolist()
print(f"\nCrypto:\n{list_crypto}")

AAPL
MSFT
NVDA
AMZN
META
GOOGL
BRK.B
GOOG
AVGO
TSLA
SPY_ETF
DAXEX_ETF
CAC40_ETF
NKY225_ETF
DJIA_ETF
EQQQ_ETF
FTSE_ETF

Equity:
['NKY225_ETF', 'AAPL', 'AMZN', 'AVGO', 'BRK.B', 'DJIA_ETF', 'GOOG', 'GOOGL', 'META', 'MSFT', 'NVDA', 'SPY_ETF', 'TSLA', 'DAXEX_ETF', 'CAC40_ETF', 'EQQQ_ETF', 'FTSE_ETF']

Crypto:
['ADAUSDT', 'AVAXUSDT', 'BNBUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETHUSDT', 'SOLUSDT', 'TRXUSDT', 'USDCUSDT', 'XRPUSDT']


**Sample Pairs**

```
Pairs: META & BNBUSDT
Correlation: 0.787

Spread ADF Statistic: -3.3381
p-value: 0.013 (Spread is stationary)

Cointegration Test Statistic: -3.6076
p-value: 0.024 (Both series are cointegrated)

In [3]:
ticker_eqt = 'META'
ticker_cpy = 'BNBUSDT'

price_pairs = data.process_pairs_series(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index())
price_pairs

,META,BNBUSDT
OpenTime,,
2024-10-14 12:30:00,593.1400,590.50
2024-10-14 12:35:00,593.5200,590.10
2024-10-14 12:40:00,594.7200,589.30
2024-10-14 12:45:00,595.3300,588.70
2024-10-14 12:50:00,596.4400,588.30
...,...,...
2025-01-17 18:35:00,614.0775,724.02
2025-01-17 18:40:00,613.7600,724.30
2025-01-17 18:45:00,614.0500,723.72


# Technical Indicators

## Trend-Following Indicators (Momentum-based)

### Exponential Moving Average (EMA)
- Places more weight on recent prices.
- Default Parameters 12, 26

In [4]:
default_values_ema = [12, 26]
list_indicators = []

for ticker in price_pairs.columns:
    for timeperiod in default_values_ema:
        list_indicators.append(
            feature_engineering.exponential_moving_average(price_pairs[ticker], ticker, timeperiod)
        )

df_ema = pd.DataFrame(list_indicators).T

In [5]:
df_ema

,META_EMA_12,META_EMA_26,BNBUSDT_EMA_12,BNBUSDT_EMA_26
OpenTime,,,,
2024-10-14 12:30:00,NaN,NaN,NaN,NaN
2024-10-14 12:35:00,NaN,NaN,NaN,NaN
2024-10-14 12:40:00,NaN,NaN,NaN,NaN
2024-10-14 12:45:00,NaN,NaN,NaN,NaN
2024-10-14 12:50:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2025-01-17 18:35:00,613.929392,613.986008,723.587291,723.477708
2025-01-17 18:40:00,613.903332,613.969266,723.696939,723.538618
2025-01-17 18:45:00,613.925896,613.975247,723.700486,723.552054


### Moving Average Convergence Divergence (MACD)

- Uses two EMAs to measure trend strength and crossovers.
- Default Parameters:
  - Fast Period: 12
  - Slow Period: 26
  - Signal Period: 9

In [6]:
fast = 12
slow = 26
signal = 9
list_indicators = []

for ticker in price_pairs.columns:
    macd, macdsignal, macdhist = feature_engineering.moving_average_convergence_divergence(
        price_pairs[ticker], ticker, fast, slow, signal
    )
    list_indicators.append(macd)

df_macd = pd.DataFrame(list_indicators).T

In [7]:
df_macd

,META_MACD,BNBUSDT_MACD
OpenTime,,
2024-10-14 12:30:00,NaN,NaN
2024-10-14 12:35:00,NaN,NaN
2024-10-14 12:40:00,NaN,NaN
2024-10-14 12:45:00,NaN,NaN
2024-10-14 12:50:00,NaN,NaN
...,...,...
2025-01-17 18:35:00,-0.056616,0.109583
2025-01-17 18:40:00,-0.065935,0.158320
2025-01-17 18:45:00,-0.049351,0.148432


## Momentum Indicators (Oscillators)

### Relative Strength Index (RSI)

- Measures overbought (>70) and oversold (<30) conditions.
- Default Parameters: 14


In [8]:
timeperiod = 14
list_indicators = []

for ticker in price_pairs.columns:
    list_indicators.append(
        feature_engineering.relative_strength_index(price_pairs[ticker], ticker, timeperiod)
    )

df_rsi = pd.DataFrame(list_indicators).T

In [9]:
df_rsi

,META_RSI_14,BNBUSDT_RSI_14
OpenTime,,
2024-10-14 12:30:00,NaN,NaN
2024-10-14 12:35:00,NaN,NaN
2024-10-14 12:40:00,NaN,NaN
2024-10-14 12:45:00,NaN,NaN
2024-10-14 12:50:00,NaN,NaN
...,...,...
2025-01-17 18:35:00,50.800371,53.412849
2025-01-17 18:40:00,48.078678,54.948980
2025-01-17 18:45:00,50.677954,51.184095
